In [177]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print("adding project dir to path:", repo_dir)
if repo_dir not in sys.path:
    sys.path = [repo_dir] + sys.path
else:
    sys.path.remove(repo_dir)
    sys.path = [repo_dir] + sys.path
    


adding project dir to path: /Users/marijnkoolen/Code/Huygens/republic-project


In [178]:
from flair.models import SequenceTagger
from flair.data import Sentence

flair_dir = f'{repo_dir}/data/embeddings/flair_embeddings/'

# load the model you trained
model_dir = '../../data/embeddings/flair_embeddings/resources/taggers/ner-tbd-DAT-train_1.0-without_flair-mini_batch_size_8/'
train_size = 1.0
layer_name = 'DAT'
use_flair = True
with_flair = 'with_flair' if use_flair else 'without_flair'

model = {}

layers = ['DAT', 'HOE', 'LOC', 'ORG', 'COM', 'PER', 'RES', 'NAM']

for layer_name in ['single_layer']:
    model_dir = f'{flair_dir}/resources/taggers/ner-tbd-{layer_name}-train_{train_size}-{with_flair}-mini_batch_size_8'

    model[layer_name] = SequenceTagger.load(f'{model_dir}/final-model.pt')



2023-04-26 00:02:32,946 SequenceTagger predicts: Dictionary with 35 tags: O, S-HOE, B-HOE, E-HOE, I-HOE, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-ORG, B-ORG, E-ORG, I-ORG, S-DAT, B-DAT, E-DAT, I-DAT, S-RES, B-RES, E-RES, I-RES, S-COM, B-COM, E-COM, I-COM, S-NAM, B-NAM, E-NAM, I-NAM, <START>, <STOP>


In [199]:
import os
from IPython.display import HTML as html_print


def get_layer_test_file(layer_name):
    gt_dir = os.path.join(repo_dir, 'ground_truth/entities/tag_de_besluiten')
    layer_gt_dir = os.path.join(gt_dir, f'flair_training_{layer_name}')
    return os.path.join(layer_gt_dir, 'test.txt')


def get_token_tag(line):
    if line == '\n':
        return '', '<S>'
    token, tag = line.strip().split(' ')
    #print(token, tag)
    return token, tag


def get_tokens_tags(test_file):
    with open(test_file, 'rt') as fh:
        tokens_tags = []
        docs = []
        for line in fh:
            if line == '\n':
                if len(tokens_tags) > 0:
                    docs.append(tokens_tags)
                tokens_tags = []
            tokens_tags.append(get_token_tag(line))
    return docs
    

def get_tag_positions(tokens_tags):
    text = ''
    tag_position = {}
    in_tag = False
    start_pos, end_pos, tag_type = None, None, None
    prev_tag_type = None
    for token, tag in tokens_tags:
        tag_type = tag[2:] if len(tag) > 2 else None
        if tag_type != prev_tag_type:
            if start_pos and prev_tag_type is not None:
                end_pos = len(text)
                tag_position[(start_pos, end_pos)] = prev_tag_type            
            start_pos = len(text) if tag_type is not None else 0
            end_pos = None
        if tag.startswith('B-'):
            tag_type = tag[2:]
            start_pos = len(text)
            end_pos = None
        elif tag == 'O':
            if start_pos and prev_tag_type is not None:
                end_pos = len(text)
                tag_position[(start_pos, end_pos)] = prev_tag_type
            start_pos, end_pos, tag_type = None, None, None
        prev_tag_type = tag_type
        text = f'{text} {token}'
    return tag_position



def read_test_file(test_file):
    docs = get_tokens_tags(test_file)
    for doc_tokens_tags in docs:
        tokens = [token for token, tag in doc_tokens_tags]
        text = ' '.join(tokens)
        tag_position = get_tag_positions(doc_tokens_tags)
        yield {'text': text, 'tag_position': tag_position, 'filename': test_file}
        
        
def get_tagged_positions(sentence):
    tagged_position = {}
    tagged_ranges = [(label.data_point.start_position, label.data_point.end_position, label.value) for label in sentence.labels]
    text = sentence.text
    for tagged_range in tagged_ranges[::-1]:
        start, end, tag_type = tagged_range
        tagged_position[(start, end)] = tag_type
    return tagged_position
            
            
def highlight_tagged_text_positions(text, tag_position):
    for start, end in sorted(tag_position.keys(), key = lambda x: x[1], reverse=True):
        tag_type = tag_position[(start, end)]
        color = LAYER_COLOR[tag_type]
        before, tag, after = text[:start], text[start:end], text[end:]
        text = f"{before}<font color='{color}'>{tag}</font>{after}"
    return f"<p>{text}</p>"


def tag_text(text, model):
    tagged_text = Sentence(text)
    model[layer].predict(tagged_text)
    return tagged_text
        
        
def highlight_tagged_text(tagged_text):
    tagged_position = get_tagged_positions(tagged_text)
    return highlight_tagged_text_positions(tagged_text.text, tagged_position)


In [181]:
from collections import defaultdict

docs = defaultdict(list)

for layer in model:
    layer_test_file = get_layer_test_file(layer)
    docs[layer] = [doc for doc in read_test_file(layer_test_file)]
    print(layer, len(docs[layer]))

single_layer 163


In [182]:

LAYER_COLOR = {
    'DAT': 'blue',
    'HOE': 'red',
    'LOC': 'green',
    'ORG': 'purple',
    'PER': 'SlateBlue',
    'RES': 'DodgerBlue',
    'COM': 'MediumSeaGreen',
    'NAM': 'Orange'
}



layer_name = 'single_layer'
texts = []

for di in range(len(docs[layer_name])):
    doc = docs[layer][di]
    sentence = Sentence(doc['text'])
    model[layer].predict(sentence)
    tagged_position = get_tagged_positions(sentence)
    #print('\nTAGGED_POSITION:', tagged_position, '\n')
    #print('\nTAG_POSITION:', doc['tag_position'], '\n')
    text = highlight_tagged_text(doc['text'], tagged_position)
    texts.append('<h2>Tagger</h2>' + text)
    #print('TAGGED:', text)
    text = highlight_tagged_text(doc['text'], doc['tag_position'])
    #print('TAG:', text)
    texts.append('<h2>Manual</h2>' + text)
    if di > 2:
        break
    
html_print('\n\n'.join(texts))

In [189]:
# source: https://nos.nl/artikel/2472853-weer-biden-tegen-trump-voor-de-amerikanen-hoeft-dat-niet-zo
raw_texts = [
    """Na maanden om de hete brij heen draaien heeft Joe Biden officieel bekendgemaakt wat iedereen al had verwacht: hij gaat op voor een tweede termijn als president.""",
    """Net als in 2020 zet Biden zichzelf neer als de beste kandidaat die het kan opnemen tegen de beweging van Donald Trump, die hij omschrijft als extremisten die verworven vrijheden willen afpakken van de gewone Amerikaan.""",
    """Als Donald Trump de Republikeinse presidentskandidaat wordt, kan de race om het Witte Huis volgend jaar zomaar een revanchewedstrijd worden. Maar de kiezer is minder enthousiast over dit déjà vu-scenario. In een nieuwe peiling van NBC News zegt 70 procent dat Biden zich niet opnieuw verkiesbaar moet stellen (de peiling werd gehouden voor de officiële bekendmaking) en 60 procent vindt dat Trump niet nog een keer presidentskandidaat moet zijn. Dit betekent niet dat de kiezer niet op hen zou stemmen, maar de metaalmoeheid neemt toe."""
]




tagged_texts = []

for text in raw_texts:
    sentence = Sentence(text)
    model[layer].predict(sentence)
    tagged_position = get_tagged_positions(sentence)
    text = highlight_tagged_text(text, tagged_position)
    tagged_texts.append(text)
    
html_print('\n\n'.join(tagged_texts))


In [191]:
raw_texts = [
    """Ik vond dit boek op een kamer van een cliënt die ik verzorgde, ik was geïntrigeerd over de titel en het liet me niet meer los. Daarom heb ik het boek tijdens mijn vakantie meegenomen. Tot nu toe heb ik daar geen spijt van gehad.""",
    """Het is het dagboek van Hendrik Groen, Hendrik woont in een verzorgingshuis in Nederland, hij heeft een paar ouderdomskwaaltjes waarover hij regelmatig een overleg heeft met zijn huisarts en geriater. Het leuke aan deze gesprekjes vond ik dat hij ze steeds op stang jaagt met de vraag om dé pil. Zonder dat hij er voorlopig gebruik van wil maken want er zijn nog genoeg leuke dingen in het leven volgens Hendrik.""",
    """Hij besluit om het anders te gaan doen. Hij verzamelt een groepje medestanders en richt de club oud-maar-nog-niet-dood op. Samen organiseren ze om de beurt een uitje. Een kookworkshop, golfen, of naar een 3D film. Iets wat de activiteitenbegeleiders van het huis niet direct aan zouden denken. Maar ook iets wat de kok en de directrice van het huis niet echt kunnen waarderen maar omdat de reglementen niet openbaar worden gemaakt niet verboden kunnen worden."""
]

tagged_texts = []

for text in raw_texts:
    sentence = Sentence(text)
    model[layer].predict(sentence)
    tagged_position = get_tagged_positions(sentence)
    text = highlight_tagged_text(text, tagged_position)
    tagged_texts.append(text)
    
html_print('\n\n'.join(tagged_texts))


In [203]:
text = """Ontfangen een missive vanden Commissaris Johan reimst, geschreven tot Paris den 3e. deses & daernevens drie bijlagen, houdende poincten van conlideratie, raeckende t' stuck vande marine; Waerop gedelibereert sijnde hebben d'Heeren van Hollandt vande voors missive ende bijlagen versocht copie die haer E. wort geaccordeert om den haren breder gecommuniceert te worden, & sal oock aen d‛andere provincien des begerende gelijcke copie gegeven worden. o: ordinaris ambassadeur in Engelandt geschre ven tot Londen den tweeden deses houdende advertentie, ende onder anderen eenige poincten van — consideratie. Waerop gedelibereert sijnde, hebben de Provincien daervan versocht copie om in den haren breder gecommuniceert te worden, die haer E wert geaccordeert."""

#text = """Ende sal mt gene voors: is bij aenschrijvinge aenden meerwelgem: heer Prince willen van Nassau bekent gemaeckt worden, met authorisatie, om bij soo verre de noot sulcx mochte comen te vereijfschen, de voors: Compagnie te peerde, ende te voet alle, off soodanigh getal van deselve, als hij sal oordeelen noodigh te hebben, bij sichte ontbieden, ende deselve provisionelijck te rengeren onder soodanige heeft-officieren, als sijn vorsel: Dooch:, ten meesten — dienste van den Lande geraden vinden zal. Des dat soo wanneer de voors: — tweede Lijste in 't gehtel bij nader deliberatie sal wesen gearresteert, deselve Compagnien wederm gerengeert ende gebruijckt sullen werden onder soodanige regemecten, ende hooft officieren, als ij deselve Lijste breeder sal wesen — geexpresseert. Ende wert den Raed van State versocht de officieren van de bovengeroerde Compagnien aen te schrijven, datse haer gereet sullen hebben te houden — om aenstonts opt ontfangen van de ordre van den meergem: heer prince willem te connen Marcheren, als mede soodanigen ordre te stellen ende die) voorsieninge doen, ten eijnde dat de voors: Compagnien tot de voors: — expeditie gebruijckt werdende in haer bandeliers met cruijt, loot, ende lonten, uijt de respective magasijnen behoorlijck voorsien mogen werden: — mitsgaders de officieren te voet aente schrijven, datse hare Compagnien hebben te formeren in twee deelen musquettiers, en een deel piecken, ende in cas — deselve daer toe eenige wapenen van Nooden, ofte te verwisselen moesten — hebben, deselve onder hare recipissen te lichten uijt de respective magasijnen, mits dat deselve op haer wederkeeren behoorlijck werden gerestitueert, dat oock de crancke, ende mmipotente soldaten, ende die althans op Zee — geemploijeert mochten sijn, uijt de andere Compagnien in deselve guarnisoen „nen, ofte daerontrent gelegen werden verwisselt, ende geremplaceert. Des wert den voorn: Raed van State, oock wijders versocht, alle de officieren — alhier in den hage sijnde, achte doen seggen, datse haer naer hunne respective Guarnisoenen sullen hebben te begeven, oock de Gouverneurs ende Commandeurs te ordonneren, datse alle de absente officieren sullen hebben te beschrijven, om in hare respective Guarnisoenen te verschijnen, ende aldaer te — verblijven tot nader ordre. Ende sullen alle de voorn: Compagnien, voor soo veel sij sullen comen te ageren, naer de expeditie wederleeren in hunne respective guarnisoenen, daer sij uijt getrocken sijn, Eijntlijck is verstaen, dat aede Provincie van Stadt Groningen ende Ommelanden, gelijck oock — aen den Gouverneur van Wesel, ende Commandeurs van Lieroort, Bellinge, wolde, ende Langhackerschans, geschreven sal worden, dat deselve sigh exactelijck willen informeren, ende doen informeren, wat bij den Bisschep van Minister — ende sijne Troupes soude mogen aengevangen, ende ondernomen werden: — gevende daer van telckens spoedige advertentie aen haer Ho:Mo: mitsgas aen den meergem: heer Prince willen van Nassau, Wijders is verstaen, dat de voors: tweede Lijste, voor soo veel aengaet de resterende Compagnien — te peerde ende te voet, daerinne gementioneert, op dewelcke noch niet en is geresolveert, andermael gestelt sal werden in handen vande opgem: heeren hare Ho:Mo: Gedepe., ende Gecome uijt den Raed van State, op nader te visiteren, examineren, ende daer van rapport te doen. Ende sal van alle het gene voors: is Extract gesonden werden aenden meerwelgem: heer Prince Willem van Nassau, om te strecken tot sijne narichtnnge, met recommandatie, omme Lomme, ende daer het behoort conrespondentie te houden, ende doen houden, oock trachten soo veel mogelijck te vernemen, wat daecontrent omgaet, ende hare ho:Mo: van tijt tot tijt advertentie daervan geven. sullende alle de depesches hier uijt resulterende werden — geexpedieert"""

tagged_text = tag_text(text, model)
highlighted_text = highlight_tagged_text(tagged_text)

html_print(highlighted_text)
